In [1]:
!pip install langchain_mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [2]:
import getpass
import os

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = 'test_langchain'
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [8]:
from langchain_mistralai import ChatMistralAI

eval_llm = ChatMistralAI(
    model="open-mistral-7b",
    temperature=0
)

In [6]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

In [27]:
import os
import requests
import time
from langchain import hub
from langsmith.evaluation import evaluate

grade_prompt_answer_accuracy = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    input_question = example.inputs["question"]
    reference = example.outputs["ground_truth"]
    prediction = example.outputs["ground_truth"] # run.outputs["answer"] - заглушка, при инференсе убрать

    llm = eval_llm
    answer_grader = grade_prompt_answer_accuracy | llm

    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]
    time.sleep(3)

    return {"key": "answer_v_reference_score", "score": score}

def evaluate_dataset_with_langsmith(predict_rag_answer, dataset_name):
    experiment_results = evaluate(
        predict_rag_answer,
        data=dataset_name,
        evaluators=[answer_evaluator],
        experiment_prefix="rag-answer-vs-reference",
    )

    return experiment_results

def get_average_score(experiment_results):
    results = experiment_results.to_pandas()
    mean = results['feedback.answer_v_reference_score'].mean()
    return mean

def predict(message):
    return message

predict_rag_answer = predict # заглушка
dataset_name = 'Validation data' # исправить название датасета

experiment_results = evaluate_dataset_with_langsmith(predict_rag_answer, dataset_name)
average_score = get_average_score(experiment_results)

print(f"Усредненная оценка: {average_score}")


Usrednennaya otsenka: 1.0


In [30]:
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """
    input_question = example.inputs["question"]
    prediction = example.outputs["ground_truth"] # run.outputs["answer"] - заглушка, при инференсе убрать

    # LLM grader
    llm = eval_llm
    answer_grader = grade_prompt_answer_helpfulness | llm

    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]
    time.sleep(3)

    return {"key": "answer_helpfulness_score", "score": score}

predict_rag_answer = predict # заглушка
dataset_name = 'Validation data' # исправить название датасета

experiment_results = evaluate_dataset_with_langsmith(predict_rag_answer, dataset_name)
average_score = get_average_score(experiment_results)

print(f"Усредненная оценка: {average_score}")

View the evaluation results for experiment: 'rag-answer-vs-reference-4d3cabcd' at:
https://smith.langchain.com/o/83698e54-c7d9-4b67-bcce-7c8d394a5542/datasets/1482ffb8-60ed-4312-ae46-7d6e0d30016d/compare?selectedSessions=a1442b31-1fa7-41ad-b39c-7ee29a2de2aa




0it [00:00, ?it/s]

Усредненная оценка: 1.0


In [ ]:
# Prompt
grade_prompt_hallucinations = prompt = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]

    prediction = run.outputs["answer"]
    llm = eval_llm

    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]
    time.sleep(3)

    return {"key": "answer_hallucination", "score": score}

# без предикта не вывезти)

In [ ]:
# Grade prompt
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]

    llm = eval_llm
    answer_grader = grade_prompt_doc_relevance | llm

    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]
    time.sleep(3)

    return {"key": "document_relevance", "score": score}

# то же самое, зависит от контекста, но запуски одинаковы